In [1]:
"""We are going to connect to the Geoserver and upload some data"""
# info source: https://pypi.org/project/geoserver-rest/

import os
from geo.Geoserver import Geoserver #make sure it is updated
import paramiko

In [2]:
"""WARNING: No hagas push de esta información"""
#Initialize the library
"""Production-geoserver"""
geo = Geoserver('https://integratedmodelling.org/dev-geoserver', username='', password='')

"""WARNING: No hagas push de esta información"""
server_ip = "192.168.250.122"
username = ""
password = ""


In [3]:
def start_ssh_client(server_ip, username, password):
    ssh_client = paramiko.SSHClient()
    ssh_client.load_system_host_keys()
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh_client.connect(server_ip, username=username, password=password)
    return ssh_client

def sudo_server_petition(ssh_client, password, petition):
    # Command to execute (This is for AKIF)
    command = f'echo {password} | sudo -S {petition}'
    # Execute the command
    stdin, stdout, stderr = ssh_client.exec_command(command)
    # Capture the output
    output = stdout.read().decode().strip()
    
    return output.split('\n') # I create a list here

def server_petition(ssh_client, petition):
    # Command to execute (This is for URA)
    command = f'{petition}'
    # Execute the command
    stdin, stdout, stderr = ssh_client.exec_command(command)
    # Capture the output
    output = stdout.read().decode().strip()
    
    return output.split('\n') # I create a list here

def start_scp_client(ssh_client):
    scp_client = ssh_client.open_sftp()
    # scp_client = scp.SCPClient(ssh_client.get_transport())
    return scp_client

def upload_file_to_server(scp_client, local_path, remote_directory):
    scp_client.put(local_path, remote_path=remote_directory)
    scp_client.close()


### Production Geoserver

In [ ]:
ssh_client = start_ssh_client(server_ip, username, password)

# Check that the geoserver_transfer is empty
read_request = r"dir /data/geoserver_transfer"
geoserver_transfer = server_petition(ssh_client, read_request)
print(geoserver_transfer)

# Pick the container name
# Esto es importante, ya que el nombre cambia ligeramente cuando se reinicia docker
read_docker_containers_request = r"docker ps --format '{{.Names}}'"
containers = server_petition(ssh_client, read_docker_containers_request)
demo_geoserver_container_list = [name for name in containers if name.startswith("demo_geoserver.")] # Cuidado que no sea el del postgis
demo_geoserver_container = str(demo_geoserver_container_list[0]) # Saca el elemento a una string
print(demo_geoserver_container)

# Optional: Get basename folders in docker container
read_docker_data_request = r"dir /data/docker-data/volumes/geoserver_data/_data/data"
geoserver_folders = server_petition(ssh_client, read_docker_data_request)
print(geoserver_folders)

In [ ]:
"""For production"""
# We asume we put everything on the same folder
local_path = r"C:\Users\admin\Downloads\dataset-satellite-land-cover\test"
remote_directory = r"\data\geoserver_transfer" 
docker_base_folder = "/opt/geoserver/data_dir/data/"
namespace_folder = "im-data-global-landcover"

for folder in os.listdir(local_path):
    # Make the folder inside the data transfer
    geoserver_transfer_folder = os.path.join(remote_directory, folder).replace(os.sep, '/')
    create_folder_request = f"mkdir {geoserver_transfer_folder}"
    server_petition(ssh_client, create_folder_request)

    # Get the files
    folder_path = os.path.join(local_path, folder)
    file_list = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
    print("Files in the folder: ", file_list)

    # Start scp and import the resources
    scp_client = start_scp_client(ssh_client)
    for file in file_list:
        # Prepare both local and remote path (Pro TIP: Always including the file name.xx into the path)
        local_file = os.path.join(folder_path, file)
        remote_file = os.path.join(geoserver_transfer_folder, file).replace(os.sep, '/') # f"{folder_destination}/{file}"
        print(remote_file)
        scp_client.put(local_file, remote_file)

    # do the docker cp to the destination
    geoserver_transfer_folder
    docker_cp_request = f"docker cp {geoserver_transfer_folder} {demo_geoserver_container}:{docker_base_folder}{namespace_folder}"
    server_petition(ssh_client, docker_cp_request)

    # Delete the folder in the geoserver_folder
    delete_folder_request = f"rm -r {geoserver_transfer_folder}"
    server_petition(ssh_client, delete_folder_request)

scp_client.close()
ssh_client.close()

### Aux geoserver

In [9]:
"""Develop way"""
ssh_client = start_ssh_client(server_ip, username, password)
# geoserver aux
# read_petittion = r"dir /data_transfer"

read_request = r"dir /samba/public/demo-geo-stack/geoserver/data/ruben"
# move_pettition = r"mv *.tif /samba/public/demo-geo-stack/geoserver/data/ruben"

result = sudo_server_petition(ssh_client, password, read_request)
print(result)

['grid.cpg  tcd_2012_020m_eu_03035_d03_full_mosaic', 'grid.dbf  tcd_2012_020m_eu_03035_d03_full_reclass.tif', 'grid.prj  tcd_2015_020m_eu_03035_d05_full_reclass_bad_projection.tif', 'grid.shp  tcd_2015_020m_eu_03035_d05_full_reclass_good_projection.tif', 'grid.shx  tcd_2018_010m_eu_03035_v2_0_mosaic']


In [11]:
read_request = r"dir /data_transfer"
read_request = r"dir /samba/public/demo-geo-stack/geoserver/data/ruben"
result = server_petition(ssh_client, read_request)
print(result)

['']


In [ ]:
"""For develop"""
# We asume we put everything on the same folder
local_path = r"C:\Users\admin\Downloads\test\super_test"
remote_directory = r"/data_transfer" 

for folder in os.listdir(local_path):
    # Make the folder inside the data transfer
    geoserver_transfer_folder = os.path.join(remote_directory, folder).replace(os.sep, '/')
    create_folder_request = f"mkdir {geoserver_transfer_folder}"
    server_petition(ssh_client, create_folder_request)

    # Get the files
    folder_path = os.path.join(local_path, folder)
    file_list = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
    print("Files in the folder: ", file_list)

    # Start scp and import the resources
    scp_client = start_scp_client(ssh_client)
    for file in file_list:
        # Prepare both local and remote path (Pro TIP: Always including the file name.xx into the path)
        local_file = os.path.join(folder_path, file)
        remote_file = os.path.join(geoserver_transfer_folder, file).replace(os.sep, '/') # f"{folder_destination}/{file}"
        print(remote_file)
        scp_client.put(local_file, remote_file)

    # move the data to the destination
    geoserver_transfer_folder
    docker_cp_request = f"docker cp {geoserver_transfer_folder} {demo_geoserver_container}:{docker_base_folder}{namespace_folder}"
    server_petition(ssh_client, docker_cp_request)

    # Delete the folder in the data_transfer
    delete_folder_request = f"rm -r {geoserver_transfer_folder}"
    server_petition(ssh_client, delete_folder_request)

scp_client.close()
ssh_client.close()

# Geoserver

In [17]:
def get_raster_file_list(path):
    File_list = [] #f for f in os.listdir(path) if os.isfile(mypath,f)
    for file in os.listdir(path):
        if file.endswith(".tif") or file.endswith(".tiff"):
            if file not in File_list:
                File_list.append(os.path.join(path,file))
        else:
            pass
    return File_list

def geoserver_raster_upload(geo, raster_file, workspace):
    # datastore_path = "/data/" + folder_name
    # datastore_path = os.path.join("data", folder_name)
    location = raster_file
    name = os.path.basename(raster_file).replace('.tif','')
    # Upload raster data to the geoserver
    geo.create_coveragestore(layer_name=name, path=location, workspace=workspace) # , data_source=datastore_path
    return print("It's over, congratulations")

In [16]:
#Input paramters
path = r"Z:\data\ruben\test"

workspace = "ruben"
datastore = "your_datastore"
folder_name = "ruben"


In [18]:
get_workspace = geo.get_workspace(workspace)
# Check if the workspace exists, if not, create it
if get_workspace is None:
    # Create a workspace if necesary
    geo.create_workspace(workspace=workspace)

raster_files = get_raster_file_list(path)



In [13]:
print(os.path.join("what", "ever"))

what\ever


In [19]:
for raster in raster_files:
    geoserver_raster_upload(geo, raster, workspace)


#geo.create_coveragestore(lyr_name="global-accessibility-2021", path="C:/Users/ruben.crespo/Documents/03_tests/global_accesibility/access_50k/store/global-accessibility-2021.tif", workspace='im-data-global-demography')

It's over, congratulations
